In this notebook we are looking to see what proprotion of prescriptions are for one week, one month, two months and three months to inform our rapid evidence review on prescription duration during a pandemic.

In order to do this we use the same methodology as the [OpenPrescribing Seven Day Prescribing aka dosette boxes measure](https://openprescribing.net/measure/seven_day_prescribing/national/england/) i.e. we select a basket of commonly prescribed medicines which are almost exclusively used once daily so we can ascertain the duration based on quantity.

In [1]:
##importing libraries that are need to support analysis
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import os

In [2]:
## here we extract data for modelling
sql = '''
SELECT
  quantity_per_item,
  sum(items) as items
FROM
 ebmdatalab.hscic.raw_prescribing_normalised AS presc
INNER JOIN
  ebmdatalab.hscic.practices AS prac
ON
  presc.practice = prac.code
WHERE
(bnf_code LIKE "0205051R0%" OR  ##ramipril
bnf_code LIKE "0212000B0%" OR ##atrovastatin
bnf_code LIKE "0212000Y0%" OR ##simvastatin
bnf_code LIKE "0602010V0%" OR ##levothyroxine
bnf_code LIKE "0206020A0%") ##amlodipine
AND
(bnf_name LIKE '%tablet%' OR
bnf_name LIKE '% tab %' OR
bnf_name LIKE '% tab' OR
bnf_name LIKE '% tabs %' OR
bnf_name LIKE '% tabs' OR
bnf_name LIKE '%capsule%' OR
bnf_name LIKE '% caps %' OR
bnf_name LIKE '% caps' OR
bnf_name LIKE '%caplet%' OR
bnf_name LIKE '%Chewtab%') ##this restricts to tablets or capsules
AND
setting = 4
AND (month BETWEEN '2019-01-01'
    AND '2019-12-01') ##this restricts to one year 2019 
GROUP BY
  quantity_per_item
    '''

df_pandemic_repeatrx = bq.cached_read(sql, csv_path=os.path.join('..','data','pandemic_repeatrx.csv'))
df_pandemic_repeatrx.head(10)

,quantity_per_item,items
0,137.0,1
1,230.0,1
2,220.0,25
3,350.0,69
4,163.0,12
5,92.0,330
6,103.0,7
7,248.0,4
8,42.0,98628
9,326.0,11


In [3]:
## here we get the total quantity on each prescription as a column
df_pandemic_repeatrx["total_quantity"] = df_pandemic_repeatrx["quantity_per_item"]*df_pandemic_repeatrx["items"] 


In [4]:
###here we make a list of common durations e.g. week, month etc
lst = [7,28,56,84]

In [5]:
##lets have a look at the common durations
df_common = df_pandemic_repeatrx.loc[(df_pandemic_repeatrx["quantity_per_item"].isin(lst))]

print(df_common)

     quantity_per_item     items  total_quantity
88                56.0  39854386    2.231846e+09
111               28.0  88728559    2.484400e+09
130                7.0  23443477    1.641043e+08
216               84.0   4838445    4.064294e+08


In [6]:
## lets see what proprotions based on the volume of tabs/caps
total = df_pandemic_repeatrx["total_quantity"].sum()
df_common["proportion_of_qty"] = df_common["total_quantity"]/total*100
df_common

<ipython-input-6-8805583853b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_common["proportion_of_qty"] = df_common["total_quantity"]/total*100


,quantity_per_item,items,total_quantity,proportion_of_qty
88,56.0,39854386,2.231846e+09,40.849652
111,28.0,88728559,2.484400e+09,45.472169
130,7.0,23443477,1.641043e+08,3.003615
216,84.0,4838445,4.064294e+08,7.438910


 45% of the total volume appears on one month prescriptions.